In [1]:
# Download required packages

%pip install requests_cache -q
%pip install tqdm
%pip install pandas
%pip install python-dotenv
%pip install requests
%pip install kalshi-python

Note: you may need to restart the kernel to use updated packages.
  Using cached tqdm-4.67.0-py3-none-any.whl.metadata (57 kB)
Using cached tqdm-4.67.0-py3-none-any.whl (78 kB)
Note: you may need to restart the kernel to use updated packages.
  Using cached pandas-2.2.3-cp313-cp313-macosx_11_0_arm64.whl.metadata (89 kB)
  Using cached numpy-2.1.3-cp313-cp313-macosx_14_0_arm64.whl.metadata (62 kB)
  Using cached pytz-2024.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.2-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.3-cp313-cp313-macosx_11_0_arm64.whl (11.3 MB)
Using cached numpy-2.1.3-cp313-cp313-macosx_14_0_arm64.whl (5.1 MB)
Using cached pytz-2024.2-py2.py3-none-any.whl (508 kB)
Using cached tzdata-2024.2-py2.py3-none-any.whl (346 kB)
Note: you may need to restart the kernel to use updated packages.
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)
Note: you may need to re

In [2]:
import requests
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("KALSHI_KEY")

event_ticker = 'KXLIVENATIONUS-30'
KALSHI_EVENTS = "https://api.elections.kalshi.com/trade-api/v2/events"
KALSHI_EVENT = f"https://api.elections.kalshi.com/trade-api/v2/events/{event_ticker}"

In [3]:
r = requests.get(KALSHI_EVENTS)
response = r.json()
response


{'events': [{'event_ticker': 'KXLIVENATIONUS-30',
   'series_ticker': 'KXLIVENATIONUS',
   'sub_title': 'Before Jan 1, 2030',
   'title': 'Courts consider Live Nation a monopoly?',
   'mutually_exclusive': False,
   'category': 'Companies'},
  {'event_ticker': 'KXAMEND22-29',
   'series_ticker': 'KXAMEND22',
   'sub_title': 'Before 2029',
   'title': 'Will Trump be allowed to run for a 3rd term?',
   'mutually_exclusive': False,
   'category': 'Politics'},
  {'event_ticker': 'KXTRUMPRESIGN',
   'series_ticker': 'KXTRUMPRESIGN',
   'sub_title': 'Before his term ends',
   'title': 'Will Trump resign before his term is up?',
   'mutually_exclusive': False,
   'category': 'Politics'},
  {'event_ticker': 'KXSCOURT-29',
   'series_ticker': 'KXSCOURT',
   'sub_title': 'By January 20, 2029',
   'title': 'Who will be the next Supreme Court justice?',
   'mutually_exclusive': True,
   'category': 'Politics'},
  {'event_ticker': 'KXPRESPARTY-2028',
   'series_ticker': 'KXPRESPARTY',
   'sub_title

In [4]:
r = requests.get(KALSHI_EVENT)
response = r.json()
response

{'event': {'event_ticker': 'KXLIVENATIONUS-30',
  'series_ticker': 'KXLIVENATIONUS',
  'sub_title': 'Before Jan 1, 2030',
  'title': 'Courts consider Live Nation a monopoly?',
  'mutually_exclusive': False,
  'category': 'Companies'},
 'markets': [{'ticker': 'KXLIVENATIONUS-30-JAN01',
   'event_ticker': 'KXLIVENATIONUS-30',
   'market_type': 'binary',
   'title': '',
   'subtitle': '',
   'yes_sub_title': 'Before Jan 1, 2030',
   'no_sub_title': 'Before Jan 1, 2030',
   'open_time': '2024-10-28T14:00:00Z',
   'close_time': '2030-01-01T15:00:00Z',
   'expected_expiration_time': '2030-01-01T15:00:00Z',
   'expiration_time': '2030-01-01T15:00:00Z',
   'latest_expiration_time': '2030-01-01T15:00:00Z',
   'settlement_timer_seconds': 300,
   'status': 'active',
   'response_price_units': 'usd_cent',
   'notional_value': 100,
   'tick_size': 1,
   'yes_bid': 7,
   'yes_ask': 64,
   'no_bid': 36,
   'no_ask': 93,
   'last_price': 65,
   'previous_yes_bid': 0,
   'previous_yes_ask': 0,
   'prev

In [12]:
import pandas as pd
from tqdm import tqdm
import pickle

# Read Kalshi Events to a DataFrame
cursor = ""
events = []

with tqdm(total=float('inf'), desc="Fetching markets") as pbar:
    while True:
        url = KALSHI_EVENTS

        response = requests.get(url, params=dict(
            limit=200,
            cursor=cursor,
            with_nested_markets=True
        ))
        r = response.json()

        # Append the new events
        events.extend(r.get("events", []))
        
        # Update cursor and break if there are no new results
        new_cursor = r.get("cursor")
        if new_cursor == cursor or not new_cursor:
            break
        cursor = new_cursor
        pbar.update(len(r.get("events", [])))

# Convert to DataFrame
events_df = pd.DataFrame(events)

# Cache into a pickle file
with open('../../data/kalshi_event_data.pkl', 'wb') as f:
    pickle.dump(events_df, f)


Fetching markets: 800it [00:05, 155.17it/s]


In [6]:
import pickle

with open('../../data/kalshi_event_data.pkl', 'rb') as f:
    df = pickle.load(f)
df

,event_ticker,series_ticker,sub_title,title,mutually_exclusive,category,markets,strike_date,strike_period
0,KXLIVENATIONUS-30,KXLIVENATIONUS,"Before Jan 1, 2030",Courts consider Live Nation a monopoly?,False,Companies,"[{'ticker': 'KXLIVENATIONUS-30-JAN01', 'event_...",NaN,NaN
1,KXAMEND22-29,KXAMEND22,Before 2029,Will Trump be allowed to run for a 3rd term?,False,Politics,"[{'ticker': 'KXAMEND22-29-JAN01', 'event_ticke...",NaN,NaN
2,KXTRUMPRESIGN,KXTRUMPRESIGN,Before his term ends,Will Trump resign before his term is up?,False,Politics,"[{'ticker': 'KXTRUMPRESIGN', 'event_ticker': '...",NaN,NaN
3,KXSCOURT-29,KXSCOURT,"By January 20, 2029",Who will be the next Supreme Court justice?,True,Politics,"[{'ticker': 'KXSCOURT-29-TC', 'event_ticker': ...",NaN,NaN
4,KXPRESPARTY-2028,KXPRESPARTY,In 2028,Which party will win the 2028 Presidential Ele...,True,Elections,"[{'ticker': 'KXPRESPARTY-2028-R', 'event_ticke...",NaN,NaN
...,...,...,...,...,...,...,...,...,...
940,KXETH-24OCT2417,KXETH,"On Oct 24, 2024","Ethereum price range on Oct 24, 2024 at 5pm EDT?",True,Crypto,"[{'ticker': 'KXETH-24OCT2417-T3319.99', 'event...",2024-10-24T21:00:00Z,NaN
941,KXBTCD-24OCT2417,KXBTCD,"On Oct 24, 2024","Bitcoin price on Oct 24, 2024 at 5pm EDT?",False,Crypto,"[{'ticker': 'KXBTCD-24OCT2417-T75749.99', 'eve...",2024-10-24T21:00:00Z,NaN
942,KXBTC-24OCT2417,KXBTC,"On Oct 24, 2024","Bitcoin price range on Oct 24, 2024 at 5pm EDT?",True,Crypto,"[{'ticker': 'KXBTC-24OCT2417-T75749.99', 'even...",2024-10-24T21:00:00Z,NaN
943,KXTNOTED-24OCT24,KXTNOTED,"On Oct 24, 2024","Treasury 10-year yield on Oct 24, 2024?",True,Financials,"[{'ticker': 'KXTNOTED-24OCT24-T4.40', 'event_t...",2024-10-24T19:25:00Z,NaN


In [8]:
result = df[df['category'] == 'Elections']

# Display the result
result

,event_ticker,series_ticker,sub_title,title,mutually_exclusive,category,markets,strike_date,strike_period
4,KXPRESPARTY-2028,KXPRESPARTY,In 2028,Which party will win the 2028 Presidential Ele...,True,Elections,"[{'ticker': 'KXPRESPARTY-2028-R', 'event_ticke...",NaN,NaN
9,KXPRESNOMR-28,KXPRESNOMR,In 2028,Who will be the Republican nominee in 2028?,True,Elections,"[{'ticker': 'KXPRESNOMR-28-VR', 'event_ticker'...",NaN,NaN
10,KXPRESNOMD-28,KXPRESNOMD,In 2028,Who will be the Democratic nominee in 2028?,True,Elections,"[{'ticker': 'KXPRESNOMD-28-RW', 'event_ticker'...",NaN,NaN
11,CONTROLS-2026,CONTROLS,In 2026,Which party will win the U.S. Senate in 2026?,True,Elections,"[{'ticker': 'CONTROLS-2026-R', 'event_ticker':...",NaN,NaN
12,CONTROLH-2026,CONTROLH,In 2026,Which party will win the House in 2026?,True,Elections,"[{'ticker': 'CONTROLH-2026-R', 'event_ticker':...",NaN,NaN
...,...,...,...,...,...,...,...,...,...
572,KXELECTIONMICALL,KXELECTIONMICALL,In 2024,AP calls Michigan before 8:00 AM Nov 6?,False,Elections,"[{'ticker': 'KXELECTIONMICALL-24NOV06-08', 'ev...",NaN,NaN
573,KXELECTIONGACALL,KXELECTIONGACALL,In 2024,AP calls Georgia before 8:00 AM Nov 6?,False,Elections,"[{'ticker': 'KXELECTIONGACALL-24NOV06-08', 'ev...",NaN,NaN
680,KXELECTION24RCP,KXELECTION24RCP,On election day,What will RCP's polling average be on election...,False,Elections,"[{'ticker': 'KXELECTION24RCP-0.7', 'event_tick...",NaN,NaN
761,KXELECTION24538,KXELECTION24538,In 2024,What will 538's polling average be?,False,Elections,"[{'ticker': 'KXELECTION24538-2.0', 'event_tick...",NaN,NaN


In [14]:
'''
ELECTION KALSHI MARKET
'''
import uuid
import kalshi_python
from kalshi_python.models import *
from pprint import pprint
import os
from dotenv import load_dotenv
import time

load_dotenv()  # Make sure this points to the correct path if .env isn't in the root directory

config = kalshi_python.Configuration()
# Comment the line below to use production
config.host = 'https://api.elections.kalshi.com/trade-api/v2'

# Create an API configuration passing your credentials.
# Use this if you want the kalshi_python sdk to manage the authentication for you.
kalshi_api = kalshi_python.ApiInstance(
    email=os.getenv("EMAIL"),
    password=os.getenv("PASSWORD"),
    configuration=config,
)

# Checks if the exchange is available.
exchangeStatus = kalshi_api.get_exchange_status()
print('Exchange status response: ')
pprint(exchangeStatus)

# Gets the data for a specific series.
seriesTicker = 'KXPRESPARTY'
seriesResponse = kalshi_api.get_series(seriesTicker)
print('\nSeries: ' + seriesTicker)
pprint(seriesResponse)

# Gets the events for a specific series.
seriesTicker = 'KXPRESPARTY'
eventsResponse = kalshi_api.get_events(series_ticker=seriesTicker)
print('\nEvents for series: ' + seriesTicker)
pprint(eventsResponse)

# Gets the data for a specific event.
eventTicker = 'KXPRESPARTY-2028'
eventResponse = kalshi_api.get_event(eventTicker)
print('\nEvent: ' + seriesTicker)
pprint(eventResponse)

marketTicker = 'KXPRESPARTY-2028-R'
marketResponse = kalshi_api.get_market(marketTicker)
print('\nMarket: ' + marketTicker)
pprint(marketResponse)

# Gets the orderbook for a specific market.
marketTicker = 'KXPRESPARTY-2028-R'
orderbookResponse = kalshi_api.get_market_orderbook(marketTicker)
print('\nOrderbook for market: ' + marketTicker)
pprint(orderbookResponse)


# Gets the market price history for a specific market. (ENDPOINT DOESN'T WORK)
# marketTicker = 'KXPRESPARTY-2028-R'
# seriesTicker = 'KXPRESPARTY'
# batchSize = 5000  # Maximum allowed by API
# periodInterval = 1 # 1m intervals
# candleStickResponse = kalshi_api.get_market_candlesticks(
#     ticker=marketTicker,
#     series_ticker=seriesTicker,
#     start_ts=int(time.time()) - (batchSize * periodInterval * 60),
#     end_ts=int(time.time()),
#     period_interval=periodInterval, 
# )
# print('\nMarket candlestick history for market: ' + marketTicker)
# pprint(candleStickResponse)


Exchange status response: 
{'exchange_active': True, 'trading_active': True}

Series: KXPRESPARTY
{'series': {'frequency': 'custom',
            'ticker': 'KXPRESPARTY',
            'title': 'Party winning presidency'}}

Events for series: KXPRESPARTY
{'cursor': '',
 'events': [{'category': 'Elections',
             'event_ticker': 'KXPRESPARTY-2028',
             'mutually_exclusive': True,
             'series_ticker': 'KXPRESPARTY',
             'strike_date': None,
             'strike_period': None,
             'sub_title': 'In 2028',
             'title': 'Which party will win the 2028 Presidential Election?'}]}

Event: KXPRESPARTY
{'event': {'category': 'Elections',
           'event_ticker': 'KXPRESPARTY-2028',
           'mutually_exclusive': True,
           'series_ticker': 'KXPRESPARTY',
           'strike_date': None,
           'strike_period': None,
           'sub_title': 'In 2028',
           'title': 'Which party will win the 2028 Presidential Election?'},
 'markets'

In [15]:
'''
REGULAR KALSHI MARKET: FED FUNDS RATE
'''
import uuid
import kalshi_python
from kalshi_python.models import *
from pprint import pprint
import os
from dotenv import load_dotenv
import time

load_dotenv()  # Make sure this points to the correct path if .env isn't in the root directory

config = kalshi_python.Configuration()
# Comment the line below to use production
config.host = 'https://trading-api.kalshi.com/trade-api/v2'

# Create an API configuration passing your credentials.
# Use this if you want the kalshi_python sdk to manage the authentication for you.
kalshi_api = kalshi_python.ApiInstance(
    email=os.getenv("EMAIL"),
    password=os.getenv("PASSWORD"),
    configuration=config,
)

# Checks if the exchange is available.
exchangeStatus = kalshi_api.get_exchange_status()
print('Exchange status response: ')
pprint(exchangeStatus)

# Gets the data for a specific series.
seriesTicker = 'FED'
seriesResponse = kalshi_api.get_series(seriesTicker)
print('\nSeries: ' + seriesTicker)
pprint(seriesResponse)

# Gets the events for a specific series.
seriesTicker = 'FED'
eventsResponse = kalshi_api.get_events(series_ticker=seriesTicker)
print('\nEvents for series: ' + seriesTicker)
pprint(eventsResponse)

# # Gets the data for a specific event.
eventTicker = 'FED-25DEC'
eventResponse = kalshi_api.get_event(eventTicker)
print('\nEvent: ' + seriesTicker)
pprint(eventResponse)

marketTicker = 'FED-25DEC-T4.25'
marketResponse = kalshi_api.get_market(marketTicker)
print('\nMarket: ' + marketTicker)
pprint(marketResponse)

# Gets the orderbook for a specific market.
marketTicker = 'FED-25DEC-T4.25'
orderbookResponse = kalshi_api.get_market_orderbook(marketTicker)
print('\nOrderbook for market: ' + marketTicker)
pprint(orderbookResponse)

# Gets the market candlestick history for a specific market. (ENDPOINT DOESN'T WORK)
# marketTicker = 'FED-23DEC-T3.00'
# seriesTicker = 'FED'
# batchSize = 5000  # Maximum allowed by API
# periodInterval = 1 # 1m intervals
# candleStickResponse = kalshi_api.get_market_candlesticks(
#     ticker=marketTicker,
#     series_ticker=seriesTicker,
#     start_ts=int(time.time()) - (batchSize * periodInterval * 60),
#     end_ts=int(time.time()),
#     period_interval=periodInterval, 
# )
# print('\nMarket candlestick history for market: ' + marketTicker)
# pprint(candleStickResponse)

Exchange status response: 
{'exchange_active': True, 'trading_active': True}

Series: FED
{'series': {'frequency': 'custom', 'ticker': 'FED', 'title': 'Fed funds rate'}}

Events for series: FED
{'cursor': '',
 'events': [{'category': 'Economics',
             'event_ticker': 'FED-25DEC',
             'mutually_exclusive': False,
             'series_ticker': 'FED',
             'strike_date': '2025-12-10T19:00:00Z',
             'strike_period': None,
             'sub_title': 'On Dec 10, 2025',
             'title': 'Fed funds rate in Dec 2025?'},
            {'category': 'Economics',
             'event_ticker': 'FED-25OCT',
             'mutually_exclusive': False,
             'series_ticker': 'FED',
             'strike_date': '2025-10-29T18:00:00Z',
             'strike_period': None,
             'sub_title': 'On Oct 29, 2025',
             'title': 'Fed funds rate in Oct 2025?'},
            {'category': 'Economics',
             'event_ticker': 'FED-25SEP',
             'mutua